# n方位抽出

北を基準にn等分した角度の値と合致した列のみを抽出します.  
ついでに時計回りにソート & [0, 360)の範囲に角度を修正します.

[使い方]  
ユーザ入力欄に風速・風向の列名, n方位を抽出したいかを順に指定します.
抽出先のcsvファイルは実行時にファイル選択ボタンから選択してください.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import glob
import os
from google.colab import files, runtime # ファイルアップロード, ダウンロード

###  ユーザ入力欄   ###
col_wind_speed = "wind_speed[m/s]" # 風速の列名
col_wind_dir = "wind_dir[deg]" # 風向の列名
n_dir = 16 # 何等分 ?

## 以下, いじらない
for f in glob.glob("*.csv"):
    os.remove(f)

print("ファイルを選択してください (複数ファイルも可)")
file_names = list(files.upload().keys()) # 複数ファイルをいっきにUPしてもOK
data = [(df, path) for df, path in zip(map(pd.read_csv, file_names), map(Path, file_names))
    if {col_wind_speed, col_wind_dir} <= set(df.columns) and path.suffix == ".csv" ]
if len(data) == 0:
    print(f"error: 列名 : {col_wind_dir} または {col_wind_dir} が存在しません")

for df, path in data:
    df[col_wind_dir] = df[col_wind_dir] % 360
    df = df[df[col_wind_dir].isin(np.linspace(0, 360, n_dir, False))]
    df = df.sort_values([col_wind_speed, col_wind_dir]) # 風向でソート
    df.reset_index(drop=True, inplace=True) # 番号を振りなおす
    output_path = f"[modified] {path.stem}_{n_dir}dirs.csv"
    df.to_csv(output_path, index=False)
    files.download(output_path)